<h1>Digit Recognizer<h1>

<h2>Introduction:</h2>

The computer vision has changed a lot in the world ranging from opening a phone with face recognition to self driving cars and all this development is result of Deep Learning.Deep Learning is going to be the future.....So,if your interested in getting started with this than
my suggestion is to take course of deep learning by Andrew NG it is an awesome course which gives all insights of deep learning...

Now in this kernel i am working on MNIST data set to build a digit Recognizer.....So let's get started.

In [ ]:
#importing all the required libraries
import numpy as np#numpy array calculations
import pandas as pd#working with dataframe
import matplotlib.pyplot as plt#for visualizing the plots
import matplotlib.image as mpimg#To view images which are of the form of numbers
import seaborn as sns#For plotting

np.random.seed(0)#To get same results whenever i do this
from sklearn.model_selection import train_test_split#For validation and checking the preformance of model
from sklearn.metrics import confusion_matrix#To see where our model doing wrong
import itertools#For efficient looping
from keras.utils.np_utils import to_categorical#one hot encoding
from keras.models import Sequential#The CNN type it have other types like Residual etc.,.
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D#Types of layers
from keras.optimizers import RMSprop,Adam#fits the filter variables,weights etc.,.
from keras.preprocessing.image import ImageDataGenerator#For data augmentation
from keras.callbacks import ReduceLROnPlateau#To make sure the we reduce learning rate if model stopped learning, upto some limit

Setting style for our plots

In [ ]:
sns.set(style='white',context='notebook',palette='deep')

<h2>Load Data</h2>

In [ ]:
#Load the data
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train.head()

As we see that we want that label as output so we will take out this from this dataframe into a separate output list

In [ ]:
y_train=train['label']
X_train=train.drop(labels=['label'],axis=1)
del train

We deleted train dataframe as it is not going to be useful anyway.So now let's test data

In [ ]:
test.head()

Test data is good in the way we wanted it so now let's see how the data is.And how many training examples are there for each output(each digit).

In [ ]:
X_train.describe()

We need to normalize the values and need to bring the mean and std to zero(for fast calculation of gradient descent)

In [ ]:
sns.countplot(y_train)

We can see that the data is well distributed So we have no problem with y_train

Now check any missing values are there.

In [ ]:
X_train.isnull().sum()

There are no missing values in training and i also checked in test data also .... there are no missing values so no problem.Now let's move to next step of normalization.

In [ ]:
X_train=X_train/255.0
test=test/255.0

The above will helps in getting faster outputs by coverging faster to the global minima.
Now we will reshape this into gray image(Add channel as 1, For RGB images we will have 3 channels).

Here as we don't know the number of images in a set We will generalise it by placing (-1 which it treats as 'n' that can be seen at run time)

In [ ]:
X_train=X_train.values.reshape(-1,28,28,1)#making an 3D array like (28*28*1) how an actual image look.
test=test.values.reshape(-1,28,28,1)

Now as the output variable is categorical values we will hot encode it into a vector.So we can apply softmax function for getting output easily.

In [ ]:
y_train=to_categorical(y_train,num_classes=10)


Now split to training and validation sets....

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.1,random_state=0)#specify random state to get same outputs

In deep learning always remember we can have large datasets most of the times so we don't split in 75% and 25% ratio rather we give a small percent like 4-10% for validating and testing and those are more than enough.

In [ ]:
g=plt.imshow(X_train[1][:,:,0])#displays image

So we tranformed the input vectors to matrices to get images like this.Now let's build our Model:-

We will use:-

1)Convolution layers:-To find features from images

2)Pooling layers:-Generalising those features(like finding global features)

3)Flatten Layers:-converts to 1D vector

4)Fully connected layers:-Like neural networks (Last layer has 10 neurons with softmax function to get output).

Other things which we use:-

relu activation:-For non linearity.

Dropout:-Reduce overfitting.

Batch Normalization:-For making mini batch gradient descent faster

My model will be:-

Conv2D->Conv2D->Maxpool->Conv2D->Conv2D->Maxpool->Flatten->Dense(Fully connected)->Dense(Softmax)->output.


In [ ]:

model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu'))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(10,activation='softmax'))


Now:-

We need to set three things:-

1)Loss function:-categorical_crossentropy(for softmax)

2)score function:-accuracy(gives how correctly we are predicting the validation set)

3)optimizer:-Adam(Uses both concepts of momentum and RMSProp)

Optimizer let's you calculate the weights and filter values to represent the features.It updates the values by following gradient descent approch.

In [ ]:
optimizer=Adam()

Now we can compile our model that means we finished our model to be use like an object after compilation.

In [ ]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

Now parameter tuning for getting Global maximum and this is done by tuning the Learning rate.What we are doing is when we had no improvement in model we will slowly decrease the learning rate until some lower boundary....

The below arguments means:-

val_accuracy:-validation set accuracy

Patience:-How many epochs we see before reducing learning rate

verbose:-shows output of each epoch(takes values of 0->shows nothing,1->shows each epoch with ======= and 2->shows each epoch with number

factor:-by which we decrease the learning rate.

min_lr:-The lower boundary till we decrease the learning rate.

In [ ]:
learning_rat=ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=1,factor=0.5,min_lr=0.001)
epochs=3
batch_size=64 #powers of two is better generally and by convention we follow this

<h2>Data Augmentation</h2>

Well we know that everyone won't write the digits with same width and in same angle with same clarity so we will perform the data augmentation techinque to make sure we can see all possibilities.

In [ ]:
datagenerated=ImageDataGenerator(rotation_range=10,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1)

we rotated by 10degrees for some images and zooming some images by 10percent, shifts heights and widths to make sure we covered all the
possibilities of images we can get and we apply this to out X_train as seen below.

In [ ]:
datagenerated.fit(X_train)

Now let's run the model for 3epochs it takes easily 30min.feel free to play with arguments and optimizers and archeticture by changing something so that you can get better in sights of what happening.

In [ ]:
digit_rec=model.fit_generator(datagenerated.flow(X_train,y_train,batch_size=batch_size),
                              epochs=epochs,validation_data=(X_val,y_val),verbose=2,steps_per_epoch=X_train.shape[0],
                              callbacks=[learning_rat])

We will now work on predictions for test set

In [ ]:
results=model.predict(test)
results=np.argmax(results,axis=1)
results=pd.Series(results,name='Label')

In [ ]:
submission=pd.concat([pd.Series(range(1,28001),name='ImageId'),results],axis=1)

In [ ]:
submission.to_csv("cnn_mnist_datagen.csv",index=False)